# 集成学习综合指南
## 引言
为了快速了解集成学习，我们举个买车例子。假如当你想购买新车时，你走到第一家车店并根据销售员的建议直接购买吗？这不太可能。

你可能会浏览一些汽车门户网站，浏览网友发布的评论并比较不同的车型，仔细比较不同车型功能和价格。你也可能会询问你的朋友和同事他们的意见。简而言之，你不会直接决定要购买哪一辆车，而是会综合考虑其他人的意见做出决定。
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/05/car_dealer-300x300.jpg)

机器学习中的集成模型基于类似的思想。它将来自多个模型的决策结合起来，以提高整体效果（比如准确率、roc，f1等）。看完本篇文章后，你将在发现：集成模型可以通过多种方式实现。

本文的目的是介绍集成学习的概念和原理，并理解使用技术的算法。为了巩固您对这个多样化主题的理解，我们将针对其中的算法，使用Python结合现实问题的实际案例研究来解释原理。

>注：本文假设你对机器学习算法有基本的了解。如果不熟悉，我建议您阅读[本文](https://www.analyticsvidhya.com/blog/2017/09/common-machine-learning-algorithms/)以熟悉这些概念。

## 目录
1. 集成学习简介
2. 基本集成技巧
    - 2.1 Max Voting
    - 2.2 Averaging
    - 2.3 Weighted Average
3. 高级集成技巧
    - 3.1 Stacking
    - 3.2 Blending
    - 3.3 Bagging
    - 3.4 Boosting
4. 基于Bagging 和 Boosting的高级算法
    - 4.1 Bagging meta-estimator
    - 4.2 Random Forest
    - 4.3 AdaBoost
    - 4.4 GBM
    - 4.5 XGB
    - 4.6 Light GBM
    - 4.7 CatBoost

## 1 集成学习简介
让我们通过一个例子来理解集成学习的概念。假设你是一名电影导演，你已经创作了一部关于非常重要且有趣的话题的短片。现在，您希望在公开之前对电影进行初步反馈（评级）。你可以用这种方式做什么？

A：您可以让一位朋友为您评分电影。
现在，你所选择的人完全有可能非常爱你，并且不想通过为你创造的可怕工作提供1星评级来打破你的心。

B：另一种方法可以是让你的5位同事评价这部电影。
这应该可以更好地了解电影。此方法可以为您的电影提供诚实的评级。但问题仍然存在。这5个人可能不是关于电影主题的“主题专家”。当然，他们可能会理解电影摄影，镜头或音频，但同时可能不是黑暗幽默的最佳裁判。

C：要求50个人评价这部电影怎么样？
其中一些可能是你的朋友，其中一些可能是你的同事，有些甚至可能是完全陌生的。

在这种情况下，答案会更加概括和多样化，因为现在你们拥有不同技能的人。事实证明 - 与我们之前看到的案例相比，这是获得诚实评级的更好方法。

通过这些示例，您可以推断出与个人相比，不同群体的人可能做出更好的决策。与单一模型相比，各种模型也是如此。机器学习的这种多样化是通过一种称为Ensemble Learning的技术实现的。

现在您已经了解了整体学习的要点 - 让我们看看整体学习中的各种技术及其实现。

## 2 简单集成技巧
在本节中，我们将介绍一些简单但功能强大的技术，即：

1. Max Voting
2. Averaging
3. Weighted Averaging

### 2.1 Max Voting
最大投票方法通常用于分类问题。在该技术中，使用多个模型来对每个数据点进行预测。每个模型的预测被视为“投票”。我们从大多数模型得到的预测被用作最终预测。

例如，当你让5位同事评价你的电影时（5分）;我们假设其中三个将它评为4，而其中两个给它一个5.由于大多数评分为4，最终评级将被视为4.您可以将此视为采用所有预测的模式。

最大投票的结果将是这样的：

| Colleague 1 | Colleague 2 | Colleague 3 |Colleague 4	|Colleague 5|	Final rating
| ------ | ------ | ------ | ------ | ------ | ------ |
|5|	4	|5	|4	|4	|4|


**数据准备**

In [20]:
import pandas
from sklearn.model_selection import train_test_split
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pandas.read_csv('pima-indians-diabetes.csv', names=names)
X=data[['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age']]
y=data['class']
x_train,x_test,y_train,y_test=train_test_split(X,y)
print(data.shape,x_test.shape)
data.head()

(768, 9) (192, 8)


,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


**实例代码**

这里x_train由训练数据中的独立变量组成，y_train是训练数据的目标变量。验证集是x_test（自变量）和y_test（目标变量）。

In [24]:
import numpy as np
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
model1 = DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3= LogisticRegression()

model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)

pred1=model1.predict(x_test)
pred2=model2.predict(x_test)
pred3=model3.predict(x_test)

final_pred = np.array([])
for i in range(0,len(x_test)):
    final_pred = np.append(final_pred, [[pred1[i], pred2[i], pred3[i]]])

G:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [27]:
from sklearn.ensemble import VotingClassifier
model1 = LogisticRegression(random_state=1)
model2 = DecisionTreeClassifier(random_state=1)
model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')
model.fit(x_train,y_train)
model.score(x_test,y_test)

G:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.734375